# 1-D MSA

In [15]:
from itertools import combinations
import math
import bisect
import sys
from skpp import ProjectionPursuitRegressor
import numpy as np
import random

# Functions for calculating Shapley Values

In [149]:
# following from https://github.com/susobhang70/shapley_value

def power_set(List):
    " Generates list of all permutations for list List"
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

def compute_shapley_values(n, v):
    """
    :param charteristic_function contains all possible permutations of the value function
    """
    tempList = list([i for i in range(n)])
    N = power_set(tempList)
    R_orderings = power_set(players)
    assert(len(characteristic_function) == len(R_orderings))
    
    shapley_values = []
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                cmod = len(j)
                Cui = j[:]
                bisect.insort_left(Cui,i)
                l = N.index(j)
                k = N.index(Cui)
                temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                           float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                shapley += temp

        cmod = 0
        Cui = [i]
        k = N.index(Cui)
        temp = float(characteristic_function[k]) * float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
        shapley += temp

        shapley_values.append(shapley)

    return shapley_values

# Testing MSA

First, run the real Shapley values. This is an example taken from the Wiki https://en.wikipedia.org/wiki/Shapley_value, demonstrating the contributions of workers (w) and bosses (o), where the value function is mp if the boss o is in the set, and 0 otherwise, where m = the number of workers in the set S and p is the profit from each worker.


First, we compute the full Shapley values.

## Full Shapley Computation

In [152]:

p = 1 # contribution of each worker

# 4 workers and 1 owner
players = np.array(['w','w','w','w','o'])
k = (players == 'w').sum()

def v(S):
    if ('o' in S):
        m = (S == 'w').sum()
        return m * p
    else:
        return 0

orderedList = list([i for i in range(len(players))])
R_orderings = power_set(orderedList)    
characteristic_function = list(map(lambda x: v(players[x]), R_orderings))

# compute the actual Shapley value
shapleys = compute_shapley_values(len(players), characteristic_function)
print("Shapley values for all workers are %s" % shapleys)

Shapley values for all workers are [0.5, 0.5, 0.5, 0.5, 2.0]


### Analyzing the reuslts

As you can see, all workers have a contribution of 0.5, and the boss ('o') has a contribution of 2.

In [153]:
print("Had to compute %i instances of the value function " % (len(R_orderings)))
print(shapleys)
assert(shapleys[0] == p/2)
assert(shapleys[4] == (k * p)/2)

Had to compute 31 instances of the value function 
[0.5, 0.5, 0.5, 0.5, 2.0]


## Computing the Predicted Shapley Values

Next, we compute the Shapley values on predicted value functions using the MSA algorithm. The main idea is that we do not need to compute all 31 instances of the value function. Instead, we will only compute 20 instances and use these to predict the missing value functions. This is useful because in our case, the value function is prediction the model and we have many TFs to predict on.

In [159]:
# Create all possible orderings of the players
orderedList = list([i for i in range(len(players))])
R_orderings = list(enumerate(power_set(orderedList)))

# choose random subset of R_orderings
sample_size = 20
R_sampled_orderings = random.sample(R_orderings, sample_size)
characteristic_function = np.zeros(len(R_orderings))

# compute the characteristic function for sampled orderings
# in our case, this will be computed from evaluating our model, building a mask from the current ordering
train_data = np.zeros((sample_size, len(players)))
y = np.zeros(sample_size)

for i, x in enumerate(R_sampled_orderings):
    y[i] = v(players[x[1]])
    for j in x[1]:
        train_data[i,j] = 1
    characteristic_function[x[0]] = y[i] 
    

# build a matrix that is #random calcs by len(players)  and predict on v
estimator = ProjectionPursuitRegressor()
estimator.fit(train_data, y)

# predict the missing v(s) 
missing_R_orderings = [x for x in R_orderings if x not in R_sampled_orderings]
for x in missing_R_orderings:
    prediction_vector = np.zeros(len(players))
    for j in x[1]:
        prediction_vector[j] = 1
    characteristic_function[x[0]] = estimator.predict(np.matrix(prediction_vector))


# compute the actual Shapley value
shapleys = compute_shapley_values(len(players), characteristic_function)

In [161]:
print("Had to compute %i instances of the value function " % (len(R_sampled_orderings)))
print(shapleys)

Had to compute 20 instances of the value function 
[0.57220564576624, 0.3890611937547275, 0.49171942194817764, 0.5518310993113194, 1.9951826392195353]
